### Topology Creation

In [2]:
from sol.topology.generators import chain_topology
t = chain_topology(5)
list(t.nodes()) # topology nodes


[0, 1, 2, 3, 4]

### Setting network resources

In [3]:
t.set_mbox(3)
t.add_service_type(3, 'firewall')
t.set_resource(3, 'cpu', 3000)
list(t.nodes(data=True))

[(0, {'resources': {}, 'services': 'switch'}),
 (1, {'resources': {}, 'services': 'switch'}),
 (2, {'resources': {}, 'services': 'switch'}),
 (3,
  {'hasMbox': 'True',
   'resources': {'cpu': 3000.0},
   'services': 'firewall;switch'}),
 (4, {'resources': {}, 'services': 'switch'})]

In [4]:
from sol import Topology
t.write_graph('mytopo.gml')
t2 = Topology('mynewtopo')
t2.load_graph('mytopo.gml')
list(t2.nodes(data=True)) # all the data is preserved

[(0, {'resources': {}, 'services': 'switch'}),
 (1, {'resources': {}, 'services': 'switch'}),
 (2, {'resources': {}, 'services': 'switch'}),
 (3,
  {'hasMbox': 'True',
   'resources': {'cpu': 3000.0},
   'services': 'firewall;switch'}),
 (4, {'resources': {}, 'services': 'switch'})]

In [5]:
for link in t.links():
    t.set_resource(link, 'bandwidth', 100)
list(t.links(data=True))

[(0, 1, {'resources': {'bandwidth': 100.0}}),
 (1, 0, {'resources': {'bandwidth': 100.0}}),
 (1, 2, {'resources': {'bandwidth': 100.0}}),
 (2, 1, {'resources': {'bandwidth': 100.0}}),
 (2, 3, {'resources': {'bandwidth': 100.0}}),
 (3, 2, {'resources': {'bandwidth': 100.0}}),
 (3, 4, {'resources': {'bandwidth': 100.0}}),
 (4, 3, {'resources': {'bandwidth': 100.0}})]

### Paths (per traffic class)

In [6]:
>>> from sol import make_tc
tc1 = make_tc(0, 4, 100)
print(tc1)

TrafficClass(tcid=0,name=,src=0,dst=4)


In [7]:
from sol.path.generate import generate_paths_tc

pptc=generate_paths_tc(t, [tc1])
print(pptc)

{TrafficClass(tcid=0,name=,src=0,dst=4): masked_array(data = [Path(nodes=[0 1 2 3 4], flowFraction=1.0)],
             mask = False,
       fill_value = ?)
}


### Builing the application and optimization

In [8]:
from sol import AppBuilder
from sol.opt.funcs import CostFuncFactory
from sol.utils.const import Objective

builder = AppBuilder()
# Create a cost function where each flow consumes 1 Mb/s regardless of traffic class
cost_func = CostFuncFactory.from_number(1)
app = builder.name('maxflowapp')\
    .pptc(pptc)\
    .objective(Objective.MAX_FLOW)\
    .add_resource('bandwidth', cost_func, 'links')\
    .build()

In [9]:
from sol import from_app, NetworkConfig, NetworkCaps

caps = NetworkCaps(t) # Create network caps from the topology
caps.add_cap('bandwidth', cap=.5) # We can use 50% of link capacities
nconfig = NetworkConfig(networkcaps=caps)
opt = from_app(t, app, nconfig) # create the optimization
opt.solve() # solve the optimization

### Examining solutions

In [12]:
opt.get_solved_objective(app) # we should see 0.5, we can route 50% of the traffic due to link constraints

0.5

In [13]:
pptc_solution = opt.get_paths()
print(pptc_solution.paths(tc1))

[Path(nodes=[0 1 2 3 4], flowFraction=0.5)]
